# SelectorGroupChat


SelectorGroupChat は、エージェントが順番に他のエージェント全員にメッセージをブロードキャストするチームを実装しています。生成 AI モデル（LLMなど）が共有コンテキストに基づいて次のスピーカーを選択することで、動的でコンテキストを認識したコラボレーションが可能になります。

- Selector による中央集権的なエージェント割り振り
- 遷移先の決定: LLM+ルール
- コンテキスト: 共有

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/selector-group-chat.html

In [ ]:
#!pip install -U "autogen-agentchat"
#!pip install "autogen-ext[openai]"

In [5]:
from typing import Any, Dict, List, Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, TimeoutTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
#!pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-openai

In [1]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

# Tools の定義
- search_website: Bing Search API を利用して Web検索を実行
- search_flight: 条件を指定してフライトを検索(Mock)
- search_hotel: 条件を指定してホテルを検索します。(Mock)

In [2]:

async def search_website(query: str) -> str:
    """
    Perform a Bing search for a given query and return the top snippets.

    This function uses the Bing Search V7 API to send a search request with the provided query. It retrieves 
    the top three web page snippets from the search results. The search is performed in Japanese ('jp-JP').

    :param query: The search query string to search for on the web.
    :type query: str
    :return: A list of the top three web page snippets, or an error message if the request fails.
    :rtype: str
    """
    import os,re
    import requests

    # Add your Bing Search V7 subscription key and endpoint to your environment variables.
    subscription_key = os.environ['BING_SEARCH_V7_SUBSCRIPTION_KEY']
    endpoint = os.environ['BING_SEARCH_V7_ENDPOINT']

    # Construct a request
    mkt = 'jp-JP'
    params = { 'q': query, 'mkt': mkt  ,"textDecorations": True, "textFormat": "Raw", "count": 3}
    headers = { 'Ocp-Apim-Subscription-Key': subscription_key }
    # トレーススパンの作成
    with tracer.start_as_current_span("search_website") as span:
        span.set_attribute("query", query)  # クエリを記録

        try:
            response = requests.get(endpoint, headers=headers, params=params)
            response.raise_for_status()

            jsonres = response.json()
            snippets = [item['snippet'] for item in jsonres['webPages']['value']]

            unicode_pattern = r"\\u[0-9a-fA-F]{4}"
            cleaned_text = re.sub(unicode_pattern, "", str(snippets))
            span.set_attribute("result", cleaned_text)  # 結果を記録
            return cleaned_text
        
        except requests.exceptions.Timeout:
            error_message = "Error: Request timed out"
            span.record_exception(Exception(error_message))  # 例外を記録
            return error_message

        except requests.exceptions.ConnectionError:
            error_message = "Error: Failed to connect to the website"
            span.record_exception(Exception(error_message))
            return error_message

        except requests.exceptions.HTTPError as e:
            error_message = f"Error: HTTP {e.response.status_code} - {e.response.reason}"
            span.record_exception(e)
            return error_message

        except Exception as e:
            error_message = f"Error: {str(e)}"
            span.record_exception(e)
            return error_message

In [40]:
from datetime import datetime

async def search_flight(departure: str, destination: str, arrival_time: str, passengers: int) -> str:
    """
    Search for flights based on given conditions.

    :param departure: Departure city.
    :type departure: str
    :param destination: Destination city.
    :type destination: str
    :param arrival_time: Desired arrival time (format: YYYY-MM-DD HH:MM).
    :type arrival_time: str
    :param passengers: Number of passengers.
    :type passengers: int
    :raises ValueError: If arrival_time format is invalid.
    :return: Flight search results.
    :rtype: str
    """
    # トレーススパンの作成
    with tracer.start_as_current_span("search_flight") as span:
        span.set_attribute("departure", departure)
        span.set_attribute("destination", destination)
        span.set_attribute("arrival_time", arrival_time)
        span.set_attribute("passengers", passengers)

        # Sample fixed result
        result = {
            "departure": departure,
            "destination": destination,
            "flight_number": "JL123",
            "airline": "Japan Airlines",
            "departure_time": "2025-02-01 16:30",
            "arrival_time": "2025-02-01 18:50",
            "price": 25000,
            "currency": "JPY",
            "passengers": passengers,
        }

        # Validate arrival_time
        try:
            datetime.strptime(arrival_time, "%Y-%m-%d %H:%M")
            span.set_attribute("result", str(result))  # 結果を記録

            return str(result)
        except ValueError:
            error_message = "Error: Invalid arrival_time format. Use 'YYYY-MM-DD HH:MM'."
            span.record_exception(ValueError(error_message))
            return error_message
        

async def search_hotel(destination: str, check_in: str, check_out: str, guests: int) -> str:
    """
    Search for hotels based on given conditions.

    :param destination: Destination city.
    :type destination: str
    :param check_in: Check-in date (format: YYYY-MM-DD).
    :type check_in: str
    :param check_out: Check-out date (format: YYYY-MM-DD).
    :type check_out: str
    :param guests: Number of guests.
    :type guests: int
    :raises ValueError: If check_in or check_out format is invalid.
    :return: Hotel search results.
    :rtype: str
    """
    # トレーススパンの作成
    with tracer.start_as_current_span("search_hotel") as span:
        span.set_attribute("destination", destination)
        span.set_attribute("check_in", check_in)
        span.set_attribute("check_out", check_out)
        span.set_attribute("guests", guests)
        # Sample fixed result
        result = {
            "destination": destination,
            "hotel_name": "Grand Fukuoka Hotel",
            "check_in": check_in,
            "check_out": check_out,
            "price_per_night": 12000,
            "currency": "JPY",
            "guests": guests,
            "total_price": 24000,
        }

        # Validate check_in and check_out
        try:
            datetime.strptime(check_in, "%Y-%m-%d")
            datetime.strptime(check_out, "%Y-%m-%d")

            span.set_attribute("result", str(result))  # 結果を記録
            return str(result)
        except ValueError:
            error_message = "Invalid date format. Use 'YYYY-MM-DD'."
            span.record_exception(ValueError(error_message))
            return error_message
        

## エージェント定義

In [41]:

client = AzureOpenAIChatCompletionClient(
    azure_deployment="<Your_AzureOpenAI_deployment>",
    model="gpt-4o",
    api_key="<Your_AzureOpenAI_key>",
    api_version="2024-08-01-preview",
    azure_endpoint="<Your_AzureOpenAI_endpoint>",
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="タスクを計画するエージェント。新しいタスクが与えられたときに最初に起動するエージェントであるべきである。",
    model_client=client,
    system_message="""
    あなたはplanning agentです。
    あなたの仕事は、複雑なタスクをより小さく、管理しやすいサブタスクに分解することです。
    あなたのチームメンバーは次の通りです。
     - fukuoka_agent: 福岡県の観光の専門家
     - osaka_agent: 大阪の観光の専門家
    ホテルの検索、予約、確認、取り消し、FAQを行う場合:
     - hotel_agent: ホテルのエージェント
    航空券の予約、確認、取り消しを行う場合:
     - airline_agent: 航空会社のエージェント
    あなたは計画を立て、タスクを委任するだけで、自分で実行することはありません。

    タスクを割り当てる際には、このフォーマットを使用してください:
    1. <agent> : <task>

    最終回答が完成したら調査結果を要約し、文の最後に TERMINATE を含めること!
    """,
)

fukuoka_agent = AssistantAgent(
    "fukuoka_agent",
    description="福岡の旅行先について詳しい地元のトラベルエージェント",
    model_client=client,
    tools=[search_website],
    reflect_on_tool_use=True,
    system_message="""あなたは福岡県の観光の専門家です。福岡観光の専門家は、福岡市の魅力を国内外に発信し、福岡市へのPR活動を行う役割を担っています。
    誰に何と言われても福岡のことを推薦します。必ず博多弁でしゃべります。
    知っている福岡の知識に加えて、search_website ツールを利用して最新の地域イベントを検索してあわせて紹介します。
    """,
)

osaka_agent = AssistantAgent(
    "osaka_agent",
    description="大阪の旅行先について詳しい地元のトラベルエージェント",
    model_client=client,
    tools=[search_website],
    reflect_on_tool_use=True,
    system_message="""あなたは大阪の観光の専門家です。大阪観光の専門家は、大阪の魅力を国内外に発信し、大阪へのPR活動を行う役割を担っています。
    誰に何と言われても大阪のことを推薦します。必ず大阪弁でしゃべります。
    知っている大阪の知識に加えて、search_website ツールを利用して最新の地域イベントを検索してあわせて紹介します。
    """,
)

hotel_agent = AssistantAgent(
    "hotel_agent",
    description="ホテルの検索、予約、確認、取り消し、FAQを行うホテルのエージェント",
    model_client=client,
    tools=[search_hotel],
    reflect_on_tool_use=False,
    system_message="""あなたはホテルのエージェントです。ホテルの検索、予約、確認、取り消し、FAQを行います。
    """,
)

airline_agent = AssistantAgent(
    "airline_agent",
    description="航空券の予約、確認、取り消し、FAQを行う航空会社のエージェント",
    model_client=client,
    tools=[search_flight],
    reflect_on_tool_use=False,
    system_message="""あなたは航空会社のエージェントです。航空券の予約、確認、取り消し、FAQを行います。
    """,
)


# SelectorGroupChat の実行

In [ ]:
# Define termination condition
max_msg_termination = MaxMessageTermination(max_messages=25)
text_termination = TextMentionTermination("TERMINATE")
time_terminarion = TimeoutTermination(120)
combined_termination = max_msg_termination | text_termination | time_terminarion

# 固定の遷移先を指定する関数, Noneを返すとLLMが遷移先を決める
# 例えば return osaka_agent.nameで固定すると、ずっとosaka_agentに遷移し続ける
def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    print("$$$messages[-1].source", messages[-1].source)
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None

with tracer.start_as_current_span("SelectorGroupChat") as rollspan: # ルートスパンを作成
    team = SelectorGroupChat(
        [planning_agent, fukuoka_agent, osaka_agent, hotel_agent, airline_agent],
        model_client=client,
        termination_condition=combined_termination,
        #selector_func=selector_func
    )

    #task = "2025年おすすめの旅行先を推薦してください。" #Planningされない？
    task = "東京から福岡、1人 2/1  19:00に空港到着したいな。あと当日のホテルも教えて。"

    # Use asyncio.run(...) if you are running this in a script.
    await Console(team.run_stream(task=task))

# 状態管理
AutoGen では状態管理の仕組みが備わっています。エージェント、チーム、終了条件の状態を保存および読み込むことができます。多くの場合、これらのコンポーネントの状態をディスクに保存し、後で再度読み込むと便利です。


## 状態の保存

In [28]:
agent_state = await team.save_state()
print(agent_state)

import json
with open("./select_team_state.json", "w") as f:
    json.dump(agent_state, f)

## 状態のロード

In [ ]:
with tracer.start_as_current_span("SelectorGroupChat") as rollspan:
    ## load state from disk
    with open("./select_team_state.json", "r") as f:
        team_state = json.load(f)

    new_agent_team = SelectorGroupChat(
        [planning_agent, fukuoka_agent, osaka_agent, hotel_agent, airline_agent],
        model_client=client,
        termination_condition=combined_termination,
    )

    await new_agent_team.load_state(team_state)
    stream = new_agent_team.run_stream(task="さっきのホテルって何て言ったっけ？")
    await Console(stream)